# Set connection with GitHub

In [1]:
# ! cd /kaggle/working && rm -rf RECsys_Challenge2024

In [2]:
from kaggle_secrets import UserSecretsClient

token = UserSecretsClient().get_secret("Token")

! git clone https://{token}@github.com/madratak/RECsys_Challenge2024.git

Cloning into 'RECsys_Challenge2024'...
remote: Enumerating objects: 1627, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 1627 (delta 3), reused 7 (delta 1), pack-reused 1618 (from 1)
Receiving objects: 100% (1627/1627), 67.08 MiB | 20.90 MiB/s, done.
Resolving deltas: 100% (948/948), done.


In [3]:
! pip install PyGithub requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 7.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 34.8 MB/s eta 0:00:00


In [4]:
! cd /kaggle/working/RECsys_Challenge2024 && python run_compile_all_cython.py

run_compile_all_cython: Found 11 Cython files in 5 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/11]: MatrixFactorizationImpressions_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1929,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorizationImpressions_Cython_Epoch.c:1252:
/opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wcpp-Wcpp]8;;]
   17 | #warning "Using deprecated NumPy API, disable it with 

In [5]:
import numpy as np
import pandas as pd
import scipy.sparse as sps
import time
import shutil

np.random.seed(42)

## Import the repository

In [6]:
from github import Github, Auth

# Authenticate using a personal access token
auth_token = Auth.Token(token)
github_client = Github(auth=auth_token)

# Define the repository name you want to find
target_repo_name = 'RECsys_Challenge2024'
repo = None

# Search for the repository in the user's repositories
try:
    for repository in github_client.get_user().get_repos():
        if repository.name == target_repo_name:
            repo = repository
            print(f"Repository '{target_repo_name}' found.")
            break
    if repo is None:
        print(f"Repository '{target_repo_name}' not found.")
except Exception as e:
    print("An error occurred while accessing the repositories:", e)


Repository 'RECsys_Challenge2024' found.


In [7]:
def upload_file(filepath_kaggle, filepath_github, commit_message):
    """
    Uploads a file from Kaggle to GitHub, updating it if it already exists in the repository,
    or creating it if it does not.

    Parameters:
    - filepath_kaggle: Path to the file in the Kaggle environment.
    - filepath_github: Target path in the GitHub repository where the file should be uploaded.
    - commit_message: Message for the commit on GitHub.
    """
    try:
        
        # Check if the file already exists in the GitHub repository
        contents = repo.get_contents(filepath_github)
        
        # If it exists, update the file
        with open(filepath_kaggle, "rb") as file:
            repo.update_file(
                contents.path, commit_message, file.read(), contents.sha
            )
        print(f"File '{filepath_github}' updated successfully.")
    
    except Exception as e:
        
        # If the file does not exist, create it
        with open(filepath_kaggle, "rb") as file:
            repo.create_file(
                filepath_github, commit_message, file.read()
            )
        print(f"File '{filepath_github}' created successfully.")

In [8]:
config = {
    'model': 'SLIMElasticNet',
    'tune_parameters': True,
    'database_path': '/kaggle/working/history_SLIMElasticNet.db',
    'copy_prev_best_params': False,
    'save_github': True
}

Import the database where previous tuning trials have been saved.

In [ ]:
try:
    shutil.copyfile(
        f'/kaggle/working/RECsys_Challenge2024/TrainedModels/' \
        f'{config["model"]}Recommender/history_{config["model"]}.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

# Construction of URM and ICM matrices

In [10]:
URM_all_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_train.csv")

n_users = len(URM_all_dataframe["user_id"].unique())
n_items = len(URM_all_dataframe["item_id"].unique())

URM_all = sps.csr_matrix((URM_all_dataframe["data"].values, 
                          (URM_all_dataframe["user_id"].values, URM_all_dataframe["item_id"].values)),
                        shape = (n_users, n_items))

URM_all

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1764607 stored elements and shape (35736, 38121)>

In [11]:
ICM_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_ICM_metadata.csv")

n_items = len(ICM_dataframe["item_id"].unique())
n_features = len(ICM_dataframe["feature_id"].unique())

ICM_all = sps.csr_matrix((ICM_dataframe["data"].values, 
                          (ICM_dataframe["item_id"].values, ICM_dataframe["feature_id"].values)),
                        shape = (n_items, n_features))

ICM_all

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2940040 stored elements and shape (38121, 94331)>

# Training

In [ ]:
! cd /kaggle/working/RECsys_Challenge2024/

/kaggle/working/RECsys_Challenge2024


In [13]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions


In [14]:
import optuna
import pandas as pd
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

def objective_function_SLIMElasticNet(optuna_trial):
    
    recommender_instance = SLIMElasticNetRecommender(URM_train)
    
    full_hyperp = {
                   "topK": optuna_trial.suggest_int("topK", 0, 1500),
                   "l1_ratio": optuna_trial.suggest_float("l1_ratio", 0.01, 1.0, log=True),
                    "alpha": optuna_trial.suggest_float("alpha", 1e-4, 1e-1, log=True),
                    "positive_only": optuna_trial.suggest_categorical("positive_only", [True, False]),
                  }        
    
    recommender_instance.fit(**full_hyperp)
    
    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)
    
    return result_df.loc[10]["MAP"]

In [15]:
if config['tune_parameters']:
    
    study = optuna.create_study(direction='maximize', study_name=f'hyperparameters_tuning_{config["model"]}',
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)

    study.optimize(objective_function_SLIMElasticNet, n_trials=5)

[I 2024-11-14 11:21:35,421] A new study created in RDB with name: hyperparameters_tuning_SLIMElasticNet


SLIMElasticNetRecommender: Processed 7205 (18.9%) in 5.00 min. Items per second: 24.01
SLIMElasticNetRecommender: Processed 14603 (38.3%) in 10.00 min. Items per second: 24.33
SLIMElasticNetRecommender: Processed 22544 (59.1%) in 15.00 min. Items per second: 25.04
SLIMElasticNetRecommender: Processed 30250 (79.4%) in 20.00 min. Items per second: 25.20
SLIMElasticNetRecommender: Processed 38121 (100.0%) in 24.79 min. Items per second: 25.62
EvaluatorHoldout: Processed 35595 (100.0%) in 31.71 sec. Users per second: 1122


[I 2024-11-14 11:46:54,980] Trial 0 finished with value: 0.055632994200627985 and parameters: {'topK': 1442, 'l1_ratio': 0.9666204269752416, 'alpha': 0.00015303659597945484, 'positive_only': False}. Best is trial 0 with value: 0.055632994200627985.


SLIMElasticNetRecommender: Processed 3800 (10.0%) in 5.00 min. Items per second: 12.66
SLIMElasticNetRecommender: Processed 7670 (20.1%) in 10.00 min. Items per second: 12.78
SLIMElasticNetRecommender: Processed 11534 (30.3%) in 15.00 min. Items per second: 12.81
SLIMElasticNetRecommender: Processed 15441 (40.5%) in 20.00 min. Items per second: 12.87
SLIMElasticNetRecommender: Processed 19280 (50.6%) in 25.00 min. Items per second: 12.85
SLIMElasticNetRecommender: Processed 23213 (60.9%) in 30.00 min. Items per second: 12.89
SLIMElasticNetRecommender: Processed 27139 (71.2%) in 35.00 min. Items per second: 12.92
SLIMElasticNetRecommender: Processed 31038 (81.4%) in 40.00 min. Items per second: 12.93
SLIMElasticNetRecommender: Processed 35040 (91.9%) in 45.00 min. Items per second: 12.98
SLIMElasticNetRecommender: Processed 38121 (100.0%) in 48.95 min. Items per second: 12.98
EvaluatorHoldout: Processed 35595 (100.0%) in 47.70 sec. Users per second: 746


[I 2024-11-14 12:36:40,275] Trial 1 finished with value: 0.05982349491746694 and parameters: {'topK': 628, 'l1_ratio': 0.033657422418087835, 'alpha': 0.0005057205459272259, 'positive_only': True}. Best is trial 1 with value: 0.05982349491746694.


SLIMElasticNetRecommender: Processed 3588 ( 9.4%) in 5.00 min. Items per second: 11.95
SLIMElasticNetRecommender: Processed 7238 (19.0%) in 10.00 min. Items per second: 12.06
SLIMElasticNetRecommender: Processed 10931 (28.7%) in 15.00 min. Items per second: 12.14
SLIMElasticNetRecommender: Processed 14739 (38.7%) in 20.00 min. Items per second: 12.28
SLIMElasticNetRecommender: Processed 18412 (48.3%) in 25.00 min. Items per second: 12.27
SLIMElasticNetRecommender: Processed 22211 (58.3%) in 30.00 min. Items per second: 12.34
SLIMElasticNetRecommender: Processed 25982 (68.2%) in 35.01 min. Items per second: 12.37
SLIMElasticNetRecommender: Processed 29680 (77.9%) in 40.01 min. Items per second: 12.36
SLIMElasticNetRecommender: Processed 33610 (88.2%) in 45.01 min. Items per second: 12.45
SLIMElasticNetRecommender: Processed 37410 (98.1%) in 50.01 min. Items per second: 12.47
SLIMElasticNetRecommender: Processed 38121 (100.0%) in 50.98 min. Items per second: 12.46
EvaluatorHoldout: Proce

[I 2024-11-14 13:28:23,928] Trial 2 finished with value: 0.059820559557810044 and parameters: {'topK': 814, 'l1_ratio': 0.10596434420029503, 'alpha': 0.00031571248849702923, 'positive_only': False}. Best is trial 1 with value: 0.05982349491746694.
[W 2024-11-14 13:29:37,894] Trial 3 failed with parameters: {'topK': 889, 'l1_ratio': 0.6910479049115915, 'alpha': 0.0004536631146119105, 'positive_only': False} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_30/4009392001.py", line 16, in objective_function_SLIMElasticNet
    recommender_instance.fit(**full_hyperp)
  File "/opt/conda/lib/python3.10/site-packages/sklearn/utils/_testing.py", line 188, in wrapper
    return fn(*args, **kwargs)
  File "/kaggle/working/RECsys_Challenge2024/Recommenders/SLIM/SLIMElasticNetRecommender.py", line 87, in fi

KeyboardInterrupt: 

In [17]:
print(optuna_study.best_trial.params)

NameError: name 'optuna_study' is not defined

In [ ]:
recommender_instance = SLIMElasticNetRecommender(URM_train + URM_validation)
recommender_instance.fit(**optuna_study.best_trial.params)

# Testing

Create the recommendations for the submission. 

In [ ]:
def create_submission(data_target_users_test, recommender_instance, cutoff=10, output_file=f"/kaggle/working/submission_{config['model']}.csv"):
    
    target_result = []

    for target in data_target_users_test["user_id"]:
        target_result.append(recommender_instance.recommend(target, cutoff=cutoff, remove_seen_flag=True))

    user_ids = data_target_users_test["user_id"]
    formatted_data = {
        "user_id": user_ids,
        "item_list": [" ".join(map(str, items)) for items in target_result]
    }

    submission_df = pd.DataFrame(formatted_data)
    submission_df.to_csv(output_file, index=False, header=["user_id", "item_list"])

    print(f"Submission file saved as {output_file}")

In [ ]:
create_submission(data_target_users_test, recommender_instance)

upload_file(
            f'/kaggle/working/submission_{config["model"]}.csv', 
            f'/kaggle/working/Recsys_Challenge_2023/TrainedModels/{config["model"]}Recommender/'\
                f'submission_{config["model"]}.csv', 
            f'New {config["model"]} submission (from kaggle notebook)'
        )

# Save Version on GitHub 

Write or import a json file where best hyperparameters are saved. 

In [ ]:
import json

if config['tune_parameters']:
    with open(f'/kaggle/working/best_params_{config["model"]}.json', 'w') as params_file:
        json.dump(study.best_params, params_file)
        
    if config['save_github']:
        upload_file(
            f'/kaggle/working/best_params_{config["model"]}.json', 
            f'/kaggle/working/RECsys_Challenge2024/TrainedModels/{config["model"]}Recommender/'\
                f'best_params_{config["model"]}.json', 
            f'{config["model"]} tuning results (from kaggle notebook)'
        )
elif config['copy_prev_best_params']:
    shutil.copyfile(
        f'/kaggle/working/RECsys_Challenge2024/TrainedModels/{config["model"]}Recommender/'\
        f'best_params_{config["model"]}.json', 
        f'/kaggle/working/best_params_{config["model"]}.json'
    )

Save the history of the tuned model.

In [ ]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'], 
        f'tuning_results/history_{config["model"]}.db',
        'Tuning db updated results (from kaggle notebook)'
    )